In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import time
import pandas as pd
import geopandas as gpd

In [2]:
# Chrome options to fix (unknown error: DevToolsActivePort file doesn't exist)
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [8]:
def click_and_scrape_posts(role, driver):
    
    jobs = []
    # Find number of posts on page
    num_posts = 0
    while True:
        try:
            driver.find_element_by_xpath(f"//*[@id='MainCol']/div[1]/ul/li[{num_posts}]")
            num_posts += 1
        except:
            break    
    print(num_posts)
    curr_post = 1
    
    while curr_post < num_posts:
        # Click on a job posting
        time.sleep(1)
        try:
            job_button = driver.find_element_by_xpath(f"//*[@id='MainCol']/div[1]/ul/li[{curr_post}]")
            job_button.click()
        except:
            print('job_button error')
            pass
        time.sleep(0.5)

        # Attempt to close Sign Up Pop Up if any
        try:
            driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span').click()
            print('closed sign up page')
        except:
            pass
        
        # Gather necessary information
        try:
            city_state_tag = driver.find_element_by_xpath(f"//*[@id='MainCol']/div[1]/ul/li[{curr_post}]/div[2]/div[2]/span")
            city_state = city_state_tag.text
            print(city_state)
        except:
            city_state = 'None'
            print('No city_state info')
        try:
            salary_range_tag = driver.find_element_by_xpath(f"//*[@id='MainCol']/div[1]/ul/li[{curr_post}]/div[2]/div[3]/div[1]/span")
            salary_range = salary_range_tag.text
        except:
            salary_range = 'None'
            print('No salary_range info')
        try:
            job_description_tag = driver.find_element_by_xpath("//*[@id='JobDescriptionContainer']")
        except:
            print('No job_description info')

        # DEBUG -- Save Info to List of Dictionaries
        try:
            html = job_description_tag.get_attribute('innerHTML')
        except:
            print('No html info')
        try:
            soup = BeautifulSoup(html)
        except:
            pass
        try:
            job_description = soup.find('div', class_='jobDescriptionContent desc').text
        except:
            job_description = 'None'
            print('No job info')
        try:
            jobs.append(
                {
                    "job_title": role,
                    "location": city_state,
                    "salary_range": salary_range,
                    "job_description": job_description
                }
            )
        except:
            print('append job error')
            pass
        
        curr_post += 1
    
    return jobs

In [4]:
def get_job_info(location, role):
    
    # Initialize variables
    driver = webdriver.Chrome()
    driver.set_window_size(1120, 1000)
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + role + '"&locKeyword=United%20States&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=false&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []
    next_page_button = driver.find_element_by_xpath('//*[@id="FooterPageNav"]/div/ul/li[7]/a')
    
    # Click the drop down menu for Date Posted
    date_posted_button = driver.find_element_by_xpath('//*[@id="filter_fromAge"]')
    date_posted_button.click()
    
    # Select Date Posted = 2 weeks
    date_range_object = driver.find_element_by_xpath('//*[@id="PrimaryDropdown"]/ul/li[5]')
    date_range_object.click()
    
    curr_page = 1
    num_pages = driver.find_element_by_xpath('//*[@id="MainCol"]/div[2]/div[1]').text.split()[-1]
    num_pages = int(num_pages)
    print(num_pages)
    
    while curr_page <= num_pages:
        print(f"Now Scraping Page {curr_page}")
        job_info = click_and_scrape_posts(role, driver)
        jobs.append(job for job in job_info)
        curr_page += 1
        next_page_button.click()
        
        
    
    # Make list of dictionaries
    output = []
    for page in range(len(jobs)):
        for job in jobs[page]:
            output.append(job)
    return output

In [6]:
jobs = get_job_info('United States', "data analyst")

30
Now Scraping Page 1
31
closed sign up page
Bloomington, IN
closed sign up page
Camden, NJ
Milwaukee, WI
United States
No salary_range info
Greensboro, GA
No salary_range info
Waukegan, IL
Pleasant Prairie, WI
Remote
No salary_range info
Orlando, FL
Orlando, FL
Covington, KY
Fargo, ND
Milwaukee, WI
Chicago, IL
Arlington, VA
Fairlawn, OH
Saint Petersburg, FL
Remote
No salary_range info
Irvington, NY
No salary_range info
North Billerica, MA
Portland, OR
Liverpool, NY
Washington, DC
Plano, TX
Fort Mill, SC
San Bernardino, CA
Boston, MA
No salary_range info
Durham, NC
Industry, CA
No salary_range info
Now Scraping Page 2
31
Houston, TX
New Rochelle, NY
Plymouth, MA
Wilmington, DE
Livermore, CA
Sacramento, CA
Monsey, NY
Newburyport, MA
No salary_range info
Plymouth, MN
Secaucus, NJ
Spartanburg, SC
Dublin, CA
Irvine, CA
San Francisco, CA
Philadelphia, PA
Phoenix, AZ
Ann Arbor, MI
No salary_range info
Norwich, CT
Danbury, CT
Tempe, AZ
No salary_range info
Denver, CO
Tampa, FL
No salary_rang

Los Alamos, NM
Culver City, CA
San Francisco, CA
Jeffersonville, IN
Hurlburt Field, FL
D'Iberville, MS
New York, NY
No salary_range info
Brockton, MA
Salt Lake City, UT
Washington, DC
Washington, DC
Indianapolis, IN
Stockton, CA
No salary_range info
Boston, MA
Chantilly, VA
No salary_range info
San Francisco, CA
Texas
No salary_range info
Los Angeles, CA
Pomona, CA
New York, NY
No salary_range info
United States
No salary_range info
Fort Washington, PA
Now Scraping Page 15
31
Richmond, VA
Washington, DC
Austin, TX
New York, NY
Los Angeles, CA
Washington, DC
Chicago, IL
Laurel, MD
Saint Louis, MO
Arlington, VA
Washington, DC
Washington, DC
Brooklyn, NY
Lavonia, GA
No salary_range info
Corvallis, OR
Gorham, NH
Eugene, OR
No salary_range info
San Francisco, CA
New York, NY
Holyoke, MA
Alexandria, VA
Hartsville, SC
Remote
No salary_range info
Remote
No salary_range info
Atlanta, GA
San Leandro, CA
No salary_range info
Durham, NC
Somerville, MA
No salary_range info
Leominster, MA
No salary_

Remote
No salary_range info
Bethesda, MD
Fort Lauderdale, FL
Remote
No salary_range info
Houston, TX
Cambridge, MA
Getzville, NY
Monrovia, CA
Phoenix, AZ
San Jose, CA
Hartford, CT
No salary_range info
Indianapolis, IN
No salary_range info
Now Scraping Page 29
31
Getzville, NY
Phoenix, AZ
San Jose, CA
Hartford, CT
No salary_range info
Indianapolis, IN
No salary_range info
Urbandale, IA
No salary_range info
Remote
No salary_range info
Remote
No salary_range info
New York, NY
Chicago, IL
United States
No salary_range info
Irvine, CA
Chesapeake, VA
Getzville, NY
Huntsville, AL
Peoria, IL
North Chicago, IL
No salary_range info
Atlanta, GA
Charlotte, NC
Kansas City, MO
No salary_range info
Tampa, FL
Remote
No salary_range info
Lisle, IL
Little Rock, AR
White Plains, NY
Sacramento, CA
Butler, PA
No salary_range info
San Antonio, TX
Santa Maria, CA
Now Scraping Page 30
31
Maple Grove, MN
No salary_range info
Quantico, VA
No salary_range info
San Diego, CA
Pittsburgh, PA
Dallas, TX
Arlington, V

In [7]:
len(jobs)

884

In [9]:
# Results of scraping 1st 30 pages of GlassDoor - Last 2 Weeks (4/29/2021)
pd.DataFrame(jobs)

,job_title,location,salary_range,job_description
0,data analyst,"Bloomington, IN",$44K - $84K (Glassdoor est.),"National Salvage, an industry leader, seeks a ..."
1,data analyst,"Camden, NJ",$65K - $131K (Glassdoor est.),"Diversity of backgrounds, ideas, thoughts, and..."
2,data analyst,"Milwaukee, WI",$65K - $79K (Employer est.),Financial Data AnalystCorporate Headquarters12...
3,data analyst,United States,None,TeamSoft’s client is looking for a Data Analys...
4,data analyst,"Greensboro, GA",None,Business Intelligence Data AnalystLocation: Re...
...,...,...,...,...
879,data analyst,"Philadelphia, PA",None,"Founded in 2007, The Oncology Institute of Hop..."
880,data analyst,North Chesterfield,None,"Title: Data Analyst\n\nLocation: Chesterfield,..."
881,data analyst,"Raleigh, NC",$42K - $85K (Glassdoor est.),JMA Pediatrics is a premier pediatric practice...
882,data analyst,"Milwaukee, WI",$45K - $80K (Glassdoor est.),"PurposeUnder general direction, review input a..."
